In [1]:
clear

#using tech with tim methods to load selenium and chromedriver

needed imporvements:
this file should be rewritten to save as the data is scraped

In [2]:
#tools to interact with the website
import selenium
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

#tools for explicit waits
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

#tools to organize data pulled from website
import pandas as pd
import io
from io import StringIO
import numpy as np
import time as time
#import libraries to set up a user agent - way for web site to contact me?
from selenium.webdriver.chrome.options import Options
#from fake_useragent import UserAgent

#assess size of objects
import sys

LOAD THE PARCEL DATA HERE

In [3]:
#used selenium_loudoun_parcel.ipynb to get the parcel data into a good format
file='C:/Users/mhcin/Loudoun_parcel.csv'
Parcel_input2=pd.read_csv(file, index_col=0, dtype = 'str')

In [4]:
#check data type
type(Parcel_input2)

pandas.core.frame.DataFrame

In [5]:
Parcel_input2

,Parcel
0,258276827000
1,584193128000
2,584196117000
3,231461150000
4,402155556000
...,...
127601,354302524000
127602,396264807000
127603,396263084000
127604,396362442000


In [6]:
#check to make sure leading zeros are preserved
Parcel_input2.iloc[8600]

Parcel    039396586000
Name: 8600, dtype: object

ACCESS WEBSITE HERE

In [7]:
PATH = "C:\Program Files (x86)\chromedriver.exe"

In [8]:
#pick the browser
#code from stackoverflow
opts = Options()
#opts.add_argument("user-agent=['tara.kniskern@gmail.com']")
driver = webdriver.Chrome(PATH, options=opts)
driver = webdriver.Chrome(PATH)

In [9]:
agent = driver.execute_script('return navigator.userAgent')

In [10]:
print(agent)

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36


#test the Parid_button, remove from final version
Parid_button = driver.find_element_by_xpath('//*[@id="ctl00_cphMainContent_SkeletonCtrl_3_txtSearchParam"]')
Parid_button.send_keys(Parcel_input2.iloc[0])

time.sleep(1)

In [ ]:
(len(Parcel_input2))

In [12]:
#set this data frame up outside of the loop 
data_taxPaid = []
data_PenPaid = []
data_IntPaid = []
data_FeesPaid = []
data_TotPaid = []

In [62]:
#def lououn_loop
for number in Parcel_input2.iloc[0:len(Parcel_input2),0]:
#for number in Parcel_input2.iloc[164:174,0]:
    #try to enter a parcel id, hopefully it will interact with this page and not reset to previous page
    if 'Parid_button' in globals():
        #reload page to avoid stale element reference
        driver.get("https://loudounportal.com/taxes/default.aspx")
        time.sleep(1)
        
    button = driver.find_element_by_xpath('//*[@id="ctl00_cphMainContent_SkeletonCtrl_3_btnAcceptRE"]')
    button.click()
    time.sleep(1)
    Parid_button = driver.find_element_by_xpath('//*[@id="ctl00_cphMainContent_SkeletonCtrl_3_txtSearchParam"]')
    Parid_button.send_keys(number)
    

    time.sleep(1)

    #hit the search button
    button_search =driver.find_element_by_xpath('//*[@id="ctl00_cphMainContent_SkeletonCtrl_3_btnSearch"]')
    button_search.click()
    
    #counter to show that the loop is making progress
    aa_idx =(Parcel_input2.loc[Parcel_input2['Parcel'] == number])
    aa = aa_idx.index[0]
    if aa % 100 == 0:
        print ('aa = {}'.format(aa))

    time.sleep(1)
    
    #need to deal with ALERT popups here because sometimes there are no records
    #attempted code from stackoverflow
    try:
        WebDriverWait(driver, 3).until(EC.alert_is_present(),
                                   'Timed out waiting for PA creation ' +
                                   'confirmation popup to appear.')

        alert = driver.switch_to.alert
        alert.accept()
        #print("alert accepted")
        #deal with no dota for that parcel, assumes forst parcel has data, will see a popup
        if 'df_taxPaid' in globals():
                df_taxPaid.iloc[0] = np.nan
                df_PenPaid.iloc[0] = np.nan
                df_IntPaid.iloc[0] = np.nan
                df_FeesPaid.iloc[0] = np.nan
                df_TotPaid.iloc[0] = np.nan 
        else:
            #this is a kludge that assumes the first parcel called has data
            print('error getting df_taxPaid')
    
    except TimeoutException:
        #print("no alert")
        #select the "view yearly payment totals"
        button_yearlytot = driver.find_element_by_xpath('//*[@id="ctl00_cphMainContent_SkeletonCtrl_3_pnlTotalResults"]/strong/div/div/div[1]/div[2]/button')
        button_yearlytot.click()

        #scrape the data - from Tech with Tim youtube tutorials
        #if the web window is too small, the website compresses the data to year and total paid. 
        #this misses a lot of data. are some homes regularly assessed fines?
        #need to write a loop in case data is missing from yearly totals button, but is in account information
        #try to find the text showing missing data, then import from account info; except - get tehe tabel information
        try:
            WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.ID, "ctl00_cphMainContent_SkeletonCtrl_3_pnlNoPmtTotals")))
                 #get the data from the table - it looks like these folks don't pay 
                #real estate taxes, lots of reason for that (e.g. veteran status)
            if 'df_taxPaid' in globals():
                    df_taxPaid.iloc[0] = '$0.00'
                    df_PenPaid.iloc[0] = '$0.00'
                    df_IntPaid.iloc[0] = '$0.00'
                    df_FeesPaid.iloc[0] = '$0.00'
                    df_TotPaid.iloc[0] = '$0.00'
            else:
                print('error getting df_taxPaid')
                #this is a kludge that assumes the first parcel called has data                                               '
        except TimeoutException:
            if 'table' in globals():
                del table
            table =  driver.find_element_by_xpath('//*[@id="ctl00_cphMainContent_SkeletonCtrl_3_gvYearlyPmtTotals"]')
            
            #the text is in string fromat, will need to change that
            #print(table.text)

            #manipulate the output so that each parcel id is a record/row, for each year
            #there will be an issue later for homes with varaible tax histories, will need to figure that out
            if 'Parcel_tax_table' in globals():
                del Parcel_tax_table
            Parcel_tax_table =table.text
           
            #wrap the string data in a Stringio wrapper - from geeeksforgeeks.org
            if 'StringData' in globals():
                del StringData
                StringData = StringIO(Parcel_tax_table)
            else:
                StringData = StringIO(Parcel_tax_table)
               
            #put into a data frame
            if 'df' in globals():
                del df
            df = pd.read_csv(StringData, sep =" ")
         

            #GET SIZE OF OUTPUT
            if 'df_size' in globals():
                del df_size
            df_size=df.shape
          
            #reshape df
            if 'df2' in globals():
                del df2
            df2 = df.iloc[0:df_size[0],0:6] #can edit this number to expected size of table
            
                      
            #change the table headers
            df2.columns= ['Year', 'Tax_Paid', 'Pen_Paid', 'Int_Paid', 'Fees_Paid','Tot_Paid']

  
            #make a data frame for each tax column. Matrix will be record parcel id in row, column header is year, 
            #make an array for tax paid, set year for index, and rotate so Parcel is rows, year is column
            if 'df_taxPaid' in globals():
                del df_taxPaid, df_PenPaid, df_IntPaid, df_FeesPaid, df_TotPaid
            df_taxPaid = df2[['Year', 'Tax_Paid']].set_index('Year').T #tax paid
            df_PenPaid = df2[['Year', 'Pen_Paid']].set_index('Year').T #penalty paid
            df_IntPaid = df2[['Year', 'Int_Paid']].set_index('Year').T #Interest paid
            df_FeesPaid = df2[['Year', 'Fees_Paid']].set_index('Year').T #fees paid
            df_TotPaid = df2[['Year', 'Tot_Paid']].set_index('Year').T #total paid: tax+penalties+fees+interest
                   
    #SHOULDNT USE APPEND OR CONCAT WTIHIN A LOOP for a dataframe. append as a list
    #RESEARCH CHUNK or using dicts
    #here will attemp to use dicts
   
    data_taxPaid.append(dict(zip(df_taxPaid.columns, df_taxPaid.iloc[0])))
    data_PenPaid.append(dict(zip(df_PenPaid.columns, df_PenPaid.iloc[0])))
    data_IntPaid.append(dict(zip(df_IntPaid.columns, df_IntPaid.iloc[0])))
    data_FeesPaid.append(dict(zip(df_FeesPaid.columns, df_FeesPaid.iloc[0])))
    data_TotPaid.append(dict(zip(df_TotPaid.columns, df_TotPaid.iloc[0])))
      #   DF_TaxPaid = pd.DataFrame(data)
    

aa = 7100
aa = 7200
aa = 8200
aa = 8400
aa = 8500
aa = 8600
aa = 8700
aa = 9000
aa = 9900


In [19]:
keys = data_taxPaid[0].keys() #or use list.data_taxPaid

In [16]:
#import tools to help save the data
import pickle
import csv

In [24]:
#WORKED - FROM STACKOVERFLOW - WRITING A LIST OF DICTIONARIES TO A CSV FILE
#save several files because sometimes the website goes offline


keys = data_taxPaid[0].keys()
with open('data_taxPaid_122302to127605.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data_taxPaid) 

In [25]:
#convert the rest of the prelim data to csv
del keys
keys = data_PenPaid[0].keys()
with open('data_PenPaid_122302to127605.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data_PenPaid) 

del keys
keys = data_IntPaid[0].keys()
with open('data_IntPaid_122302to127605.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data_IntPaid) 

del keys
keys = data_FeesPaid[0].keys()
with open('data_FeesPaid_122302to127605.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data_FeesPaid) 

del keys
keys = data_TotPaid[0].keys()
with open('data_TotPaid_122302to127605.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data_TotPaid) 

In [20]:
#closes the web browser
driver.quit()